In [7]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ustlb29z
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ustlb29z
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image

class HistopathologyDataset(Dataset):
    def __init__(self, image_dir, label_dict, preprocess):
        self.image_dir = image_dir
        self.image_files = list(label_dict.keys())
        self.labels = list(label_dict.values())
        self.preprocess = preprocess

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = self.preprocess(Image.open(img_path).convert("RGB"))
        label = self.labels[idx]
        return image, label


In [ ]:
import torch
import torch.nn as nn
import clip

# ✅ Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Load CLIP ResNet-50 model
clip_model, preprocess = clip.load("RN50", device=device)

# ✅ Define classifier with correct input dimension (1024 for RN50)
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.image_encoder = clip_model.visual
        self.classifier = nn.Linear(1024, 2)  # 2 classes: e.g., benign vs malignant

    def forward(self, images):
        with torch.no_grad():  # freeze encoder
            features = self.image_encoder(images)
        return self.classifier(features)

# ✅ Example: Initialize model
model = CLIPClassifier(clip_model).to(device)


100%|███████████████████████████████████████| 244M/244M [00:37<00:00, 6.76MiB/s]


In [ ]:
import os

def create_label_dict(base_dir):
    label_dict = {}
    class_to_idx = {}
    current_label = 0

    # Traverse benign and malignant directories
    for category in ["benign", "malignant"]:
        category_path = os.path.join(base_dir, category)
        for subfolder in os.listdir(category_path):
            class_path = os.path.join(category_path, subfolder)
            if os.path.isdir(class_path):
                class_to_idx[subfolder] = current_label
                for file in os.listdir(class_path):
                    if file.lower().endswith((".png", ".jpg", ".jpeg")):
                        label_dict[file] = current_label
                current_label += 1

    return label_dict, class_to_idx

# Set your paths
train_path = "/content/drive/MyDrive/Colab Notebooks/sample/TRAIN"
valid_path = "/content/drive/MyDrive/Colab Notebooks/sample/VALID"

train_labels, train_classes = create_label_dict(train_path)
val_labels, val_classes = create_label_dict(valid_path)


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class HistopathologyDataset(Dataset):
    def __init__(self, image_dir, label_dict, preprocess):
        self.image_dir = image_dir
        self.label_dict = label_dict
        self.preprocess = preprocess
        self.image_files = list(label_dict.keys())

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        label = self.label_dict[img_name]

        # Search both benign and malignant subdirs
        for category in ["benign", "malignant"]:
            for subfolder in os.listdir(os.path.join(self.image_dir, category)):
                image_path = os.path.join(self.image_dir, category, subfolder, img_name)
                if os.path.exists(image_path):
                    image = Image.open(image_path).convert("RGB")
                    image = self.preprocess(image)
                    return image, label

        raise FileNotFoundError(f"{img_name} not found in {self.image_dir}")


In [ ]:
!pip install open-clip-torch


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# ===== TRAIN Folder =====
train_path = '/content/drive/MyDrive/Colab Notebooks/sample/TRAIN'

print("📂 Scanning TRAIN folder...")
for root, dirs, files in os.walk(train_path):
    for fname in files:
        if fname.lower().endswith('.png'):
            try:
                img_path = os.path.join(root, fname)
                img = Image.open(img_path)

                plt.imshow(img)
                plt.title(f"TRAIN - {fname}")
                plt.axis('off')
                plt.show()

            except Exception as e:
                print(f"❌ Error loading TRAIN image {img_path}: {e}")

# ===== VALID Folder =====
valid_path = '/content/drive/MyDrive/Colab Notebooks/sample/VALID'

print("\n📂 Scanning VALID folder...")
for root, dirs, files in os.walk(valid_path):
    for fname in files:
        if fname.lower().endswith('.png'):
            try:
                img_path = os.path.join(root, fname)
                img = Image.open(img_path)

                plt.imshow(img)
                plt.title(f"VALID - {fname}")
                plt.axis('off')
                plt.show()

            except Exception as e:
                print(f"❌ Error loading VALID image {img_path}: {e}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import open_clip
import torch
import torch.nn as nn


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pre-trained CLIP
model, _, preprocess_clip = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='laion2b_s34b_b79k',
    device=device
)

# Use only the visual encoder
image_encoder = model.visual


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
# Freeze CLIP encoder
for param in image_encoder.parameters():
    param.requires_grad = False


In [ ]:
class CLIPClassifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super(CLIPClassifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.LayerNorm(512),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.encoder(x)            # Output: [B, 512]
        x = self.classifier(x)         # Output: [B, num_classes]
        return x

num_classes = len(train_classes)  # 8 for your case (4 benign + 4 malignant)
model = CLIPClassifier(image_encoder, num_classes).to(device)


In [ ]:
from torch import optim
import torch.nn.functional as F

optimizer = optim.AdamW(model.classifier.parameters(), lr=1e-4)

def train_one_epoch(model, dataloader):
    model.train()
    total_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [ ]:
from torch.utils.data import Dataset
import os
from PIL import Image

class HistologyCLIPDataset(Dataset):
    def __init__(self, root_dir, preprocess, classnames=None):
        self.root_dir = root_dir
        self.preprocess = preprocess  # CLIP image transform
        self.image_paths = []
        self.labels = []
        self.classnames = sorted(os.listdir(root_dir)) if classnames is None else classnames

        for label, classname in enumerate(self.classnames):
            class_dir = os.path.join(root_dir, classname)
            for fname in os.listdir(class_dir):
                if fname.lower().endswith('.png'):
                    self.image_paths.append(os.path.join(class_dir, fname))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.preprocess(image)  # Resize, normalize
        return image, label


In [ ]:
import os

train_path = "/content/drive/MyDrive/Colab Notebooks/sample/TRAIN"
valid_path = "/content/drive/MyDrive/Colab Notebooks/sample/VALID"

print(f"Train dataset size: {len(os.listdir(train_path))}")
print(f"Valid dataset size: {len(os.listdir(valid_path))}")


Train dataset size: 3
Valid dataset size: 3


In [ ]:
!pip install torch torchvision ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-yf3djdk7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-yf3djdk7
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import clip

class HistologyCLIPDataset(Dataset):
    def __init__(self, root_dir, preprocess, classnames):
        self.root_dir = root_dir
        self.preprocess = preprocess
        self.classnames = classnames
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(classnames):
            sub_path = os.path.join(root_dir, class_name)
            for img_file in os.listdir(sub_path):
                if img_file.endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(sub_path, img_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = self.preprocess(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]
        return image, label


In [ ]:
import torch
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


In [ ]:
train_root = "/content/drive/MyDrive/Colab Notebooks/sample/TRAIN"
classnames = []

# Get subfolder names recursively (2 levels: benign/adenosis)
for class_folder in sorted(os.listdir(train_root)):
    path = os.path.join(train_root, class_folder)
    if os.path.isdir(path):
        for subfolder in sorted(os.listdir(path)):
            classnames.append(f"{class_folder}/{subfolder}")


In [ ]:
train_dataset = HistologyCLIPDataset(train_root, preprocess, classnames)
valid_dataset = HistologyCLIPDataset("/content/drive/MyDrive/Colab Notebooks/sample/VALID", preprocess, classnames)

from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/sample/VALID/benign/Fibroadenoma'

In [ ]:
import torch.nn as nn

# Freeze CLIP
for param in model.parameters():
    param.requires_grad = False

# Replace final head with custom classifier
class CLIPCustomClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPCustomClassifier, self).__init__()
        self.clip = clip_model.visual
        self.classifier = nn.Linear(self.clip.output_dim, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.clip(x)
        return self.classifier(features)

model_custom = CLIPCustomClassifier(model, num_classes=len(classnames)).to(device)


In [ ]:
import torch.nn as nn

# Freeze CLIP
for param in model.parameters():
    param.requires_grad = False

# Replace final head with custom classifier
class CLIPCustomClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPCustomClassifier, self).__init__()
        self.clip = clip_model.visual
        self.classifier = nn.Linear(self.clip.output_dim, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.clip(x)
        return self.classifier(features)

model_custom = CLIPCustomClassifier(model, num_classes=len(classnames)).to(device)


In [ ]:
import torch.optim as optim
import torch.nn.functional as F

optimizer = optim.Adam(model_custom.classifier.parameters(), lr=1e-4)
num_epochs = 10

for epoch in range(num_epochs):
    model_custom.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        logits = model_custom(images)
        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


In [ ]:
model_custom.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_custom(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import clip

# ----------------------------
# CONFIG
# ----------------------------
train_dir = "/content/drive/MyDrive/Colab Notebooks/sample/TRAIN"
valid_dir = "/content/drive/MyDrive/Colab Notebooks/sample/VALID"
batch_size = 4
num_epochs = 25
learning_rate = 1e-4
model_save_path = "best_model.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------
# CLASS NAMES: nested folders
# ----------------------------
classnames = []
for top in sorted(os.listdir(train_dir)):
    top_path = os.path.join(train_dir, top)
    if os.path.isdir(top_path):
        for sub in sorted(os.listdir(top_path)):
            if os.path.isdir(os.path.join(top_path, sub)):
                classnames.append(f"{top}/{sub}")

# ----------------------------
# Dataset class
# ----------------------------
class HistologyCLIPDataset(Dataset):
    def __init__(self, root_dir, preprocess, classnames):
        self.root_dir = root_dir
        self.preprocess = preprocess
        self.classnames = classnames
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(classnames):
            folder = os.path.join(root_dir, class_name)
            if os.path.isdir(folder):
                for file in os.listdir(folder):
                    if file.endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(os.path.join(folder, file))
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = self.preprocess(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]
        return image, label

# ----------------------------
# Load CLIP model
# ----------------------------
model_clip, preprocess = clip.load("ViT-B/32", device=device)

# Freeze CLIP
for param in model_clip.parameters():
    param.requires_grad = False

# ----------------------------
# Custom classifier model
# ----------------------------
class CLIPCustomClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPCustomClassifier, self).__init__()
        self.clip = clip_model.visual
        self.classifier = nn.Linear(self.clip.output_dim, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.clip(x)
        return self.classifier(features)

model = CLIPCustomClassifier(model_clip, len(classnames)).to(device)

# ----------------------------
# Dataset & DataLoaders
# ----------------------------
train_dataset = HistologyCLIPDataset(train_dir, preprocess, classnames)
valid_dataset = HistologyCLIPDataset(valid_dir, preprocess, classnames)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# ----------------------------
# Optimizer & Training Setup
# ----------------------------
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
best_val_acc = 0
patience = 3
wait = 0

# ----------------------------
# Training Loop
# ----------------------------
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"\nEpoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}")

    # ----------------------------
    # Validation
    # ----------------------------
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), model_save_path)
        print(f"✅ Saved best model (Accuracy: {val_acc:.2f}%)")
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping triggered.")
            break

print("Training completed. Best validation accuracy:", best_val_acc)


In [ ]:
model.load_state_dict(torch.load("best_model.pt"))
model.eval()


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


In [ ]:
best_val_acc = 0
wait = 0
patience = 3  # stop after 3 epochs with no improvement

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"\nEpoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), "best_model.pt")
        print(f"✅ Best model saved with {val_acc:.2f}% accuracy")
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping triggered.")
            break


In [ ]:
train_losses = []
val_accuracies = []


In [ ]:
train_losses.append(total_loss)
val_accuracies.append(val_acc)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy', color='green')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid()
plt.legend()

plt.tight_layout()
plt.savefig("training_curves.png")  # Save the plot
plt.show()


In [ ]:
class HistologyCLIPDataset(Dataset):
    def __init__(self, root_dir, preprocess, classnames):
        self.root_dir = root_dir
        self.preprocess = preprocess
        valid_dataset.classnames
        self.classes = classnames     # ✅ This is what classification_report needs
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classnames)}

        self.image_paths = []
        self.labels = []

        for class_name in classnames:
            class_path = os.path.join(root_dir, class_name)
            for root, _, files in os.walk(class_path):
                for file in files:
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(os.path.join(root, file))
                        self.labels.append(self.class_to_idx[class_name])


In [ ]:
class HistologyCLIPDataset(Dataset):
    def __init__(self, root_dir, preprocess, classnames):
        self.root_dir = root_dir
        self.preprocess = preprocess
        self.classnames = classnames  # ✅ Provided list like ["benign", "malignant"]
        self.classes = classnames     # ✅ This is what classification_report needs
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classnames)}

        self.image_paths = []
        self.labels = []

        for class_name in classnames:
            class_path = os.path.join(root_dir, class_name)
            for root, _, files in os.walk(class_path):
                for file in files:
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(os.path.join(root, file))
                        self.labels.append(self.class_to_idx[class_name])


In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Ensure model is in evaluation mode
model.eval()

# Create validation loader
val_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

# Containers for predictions and labels
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 📊 Classification Report
print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=valid_dataset.classnames))

# 📉 Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
